In [1]:
%run CurveSetup.ipynb

# Extra Packages
import re
import scipy.optimize as opt
from lmfit import minimize, Parameters, Parameter, report_fit
from itertools import compress
import seaborn as sns
import copy

In [2]:
'''TEST 3: HIGHAM AND MAO
=========================='''
def fNHighamMaoCurve(forwardCurve, alpha1, alpha2, sigma1, sigma2, rho, eta, delta, timeStepsPerYear):
    timeSteps = len(forwardCurve) * timeStepsPerYear
    dt = 1/timeSteps
  
    # Set the forward Curve
    curve = np.array(forwardCurve)
    
    # Define the volatility terms
    alpha1Exponential = sigma1 * np.exp(-alpha1 * np.arange(1, len(forwardCurve)+1))
    alpha2Exponential = sigma2 * np.exp(-alpha2 * np.arange(1, len(forwardCurve)+1))
    sigma1T =  np.sqrt(1 - np.power(rho, 2))* alpha1Exponential
    sigma2T = alpha2Exponential + rho * alpha1Exponential
    
    # Terminal Curve
    terminalCurve = []
    gaussians = []
    deflateurs = []
      
    # Loop to construct curves
    for t in range(1, timeSteps+1):
        # Generate Gaussians
        rand1 = np.random.standard_normal(len(curve))
        rand2 = np.random.standard_normal(len(curve))
        
        # Calculate Drift (mu for each Brownian)
        mu1 = np.array(sigma1T * np.power(curve + delta, eta) * 
                             np.cumsum(sigma1T* np.power(curve + delta, eta)/
                                   (1 + curve)))       
        mu2 = np.array(sigma2T * np.power(curve + delta, eta) * 
                             np.cumsum(sigma2T* np.power(curve + delta, eta)/
                                   (1 + curve)))
        mu = mu1 + mu2
                       
        # Construct Curve     
        curve = (curve
                    + np.power(np.absolute(curve) + delta, eta) * dt * mu
                    + sigma1T*np.power(np.absolute(curve)+delta, eta)*sqrt(dt)*rand1 
                    + sigma2T*np.power(np.absolute(curve)+delta, eta)*sqrt(dt)*rand2)
        
        # At end of each year append the curve to TerminalCurve
        if (t%timeStepsPerYear) == 0 :
            terminalCurve.append(curve)
            deflateurs.append(1/(1+curve[0]))
            curve = curve[1:] # Delete 1 year from curve
            sigma1T = sigma1T[:len(sigma1T)-1]
            sigma2T = sigma2T[:len(sigma2T)-1]
            gaussians.append(np.matrix([stats.norm.cdf(rand1), stats.norm.cdf(rand2)]))
            
    return([terminalCurve, gaussians, deflateurs])

In [ ]:
'''INPUT PARAMETERS
====================='''
# obtained from the calibration step
forwardCurve = np.array(forwardCurves.loc['2019-12-31'])
delta = 0.5
eta = 0.99  
sigma1 = 0.3   
sigma2 = 0.3 
alpha1 = 1000 
alpha2 =  1000 
rho = 1 
timeStepsPerYear =  1
expiries = np.arange(1,len(forwardCurve)+1)
np.random.seed(50)

'''SIMULATION
==============='''
start = time()
simulatedCurves250 = []
uniforms250 = []
deflateurs = []

for i in range(1000):
    results = fNHighamMaoCurve(forwardCurve,  alpha1, alpha2, sigma1, sigma2, rho, eta, delta, timeStepsPerYear)  
    
    # Simulate
    simulatedCurves250.append(results[0])
    uniforms250.append(results[1])
    deflateurs.append(results[2])

    
'''ZERO COUPON SIMULATION
==========================='''
simulatedZeroCoupons = copy.deepcopy(simulatedCurves250)

for i in range(len(simulatedCurves250)):
    for j in range(len(simulatedCurves250[0])):
        simulatedZeroCoupons[i][j] = np.cumprod(1/1+simulatedCurves250[i][j])
        
end = time()
(end- start)/60

In [ ]:
'''FORWARD SWAP RATE PROJECTION
=================================
- For more robust results, we select swaptions from expiry 5X5 onwards.
- Maximum expiry X tenor is 30X30
- From 10Y, we space out the deltaExpiry and deltaTenor by 5Y ie 10, 15, 20, 25, 30 
'''

# Create the swaption matrix
allowedExpiryTenors = np.concatenate((np.arange(5, 10), 
                               np.arange(10, 31, 5)))
expiryMin = 5
expiryMax = 10
tenorMin = 5
tenorMax = 10

expMinIndex = np.where(allowedExpiryTenors == expiryMin)[0][0]
expMaxIndex = np.where(allowedExpiryTenors == expiryMax)[0][0]
tenMinIndex = np.where(allowedExpiryTenors == tenorMin)[0][0]
tenMaxIndex = np.where(allowedExpiryTenors == tenorMax)[0][0]

# Construct the final swaption matrix
swaptionMatrix = list(itertools.product(allowedExpiryTenors[expMinIndex: expMaxIndex+1], 
                                  allowedExpiryTenors[expMinIndex: expMaxIndex+1]))
tenors = [elem[0] for elem in swaptionMatrix]
expiries =[elem[1] for elem in swaptionMatrix]

In [ ]:
def forwardSwapRateMartingaleTest(evalDate, expiry, tenor, simulation):
    num = (simulatedZeroCoupons[simulation][evalDate][int(expiry-1)] - 
           simulatedZeroCoupons[simulation][evalDate][int(expiry+tenor-1)])
    
    PVBP = sum(simulatedZeroCoupons[simulation][evalDate][int(expiry):int(expiry+tenor-1)])
    
    return(num)

# Vectorize the function
forwardSwapRateMartingaleTestVect = np.vectorize(forwardSwapRateMartingaleTest)

# Obtain Swaption Strikes
simulatedStrikes = []
for i in range(len(simulatedZeroCoupons)):
    simulatedStrikes.append([forwardSwapRateMartingaleTestVect(np.arange(elem[0]), elem[0] - 
                                np.arange(elem[0]), elem[1], i)
                                     for elem in swaptionMatrix])

In [ ]:
'''SIGMA ALPHA BETA FUNCTION
============================='''
def sigmaAlphaBeta(evalDate, expiry, tenor, sigma1, sigma2, alpha1, alpha2, rho, eta, delta, simulation):
    # Find Zero Coupon Price at each date and calculate PVBP
    PVBP = np.sum(simulatedZeroCoupons[simulation][evalDate][expiry:tenor+expiry-1])

    # Calculate the forward swap rate
    swapForward = forwardSwapRateMartingaleTest(evalDate, expiry, tenor, simulation)

    # Calculate the ZC bond value at T_alpha and T_beta
    zcAlpha = simulatedZeroCoupons[simulation][evalDate][expiry-1]
    zcBeta = simulatedZeroCoupons[simulation][evalDate][expiry+tenor-1]

    # Calculate delta weights
    '''To be confirmed'''
    deltaWeights = [(swapForward/(1 + simulatedCurves250[simulation][evalDate][i-1]))* 
                    ((zcBeta/(zcAlpha-zcBeta)) + 
                     np.sum(simulatedCurves250[simulation][evalDate][i:expiry+tenor-1])/PVBP)
                    for i in list(range(expiry, expiry+tenor))]

    # Calculate Alpha Exponential 
    alpha1Exponential = np.exp(-alpha1 * np.arange(1, tenor + expiry))
    alpha2Exponential = np.exp(-alpha2 * np.arange(1, tenor + expiry))

    # Upsilon Terms (From volatility parametrization)
    upsilon1Terms = sqrt(1 - pow(rho, 2)) * sigma2 * alpha1Exponential 
    upsilon2Terms = sigma2 * alpha2Exponential + rho * sigma1 * alpha1Exponential

    # Combine vectors
    deltaWeightsCombination = list(itertools.combinations_with_replacement(deltaWeights, 2))
    forwardsCombination = list(itertools.combinations_with_replacement(simulatedCurves250[simulation][evalDate]
                                                                       [expiry:expiry+tenor], 2))

    # Find the product of vectors
    deltaWeightsProduct = [np.product(elem) for elem in deltaWeightsCombination]
    forwardsProduct = [np.product(elem) for elem in forwardsCombination]

    # Define indices
    indices = list(itertools.combinations_with_replacement(np.arange(expiry, 
                                                        tenor + expiry), 2))

    #Integral of terms
    sigma1Integrals = [np.dot(upsilon1Terms[max(i, j) - min(i, j):max(i, j)], upsilon1Terms[:min(i, j)])
     if i != j 
     else np.dot(upsilon1Terms[:i], upsilon1Terms[:i]) for (i,j) in indices ]

    sigma2Integrals = [np.dot(upsilon2Terms[max(i, j) - min(i, j):max(i, j)], upsilon2Terms[:min(i, j)])
     if i != j 
     else np.dot(upsilon2Terms[:i], upsilon2Terms[:i]) for (i,j) in indices ]

    # Calculate different weights
    s1 = sum(np.multiply(np.multiply(deltaWeightsProduct, forwardsProduct), sigma1Integrals))
    s2 = sum(np.multiply(np.multiply(deltaWeightsProduct, forwardsProduct), sigma2Integrals))
    
    return((s2 + s1)/ np.power(swapForward + delta, 2*eta))

# Vectorize Function
sigmaAlphaBetaVect = np.vectorize(sigmaAlphaBeta)

In [ ]:
''' Chi Square Pricer - Payer
=============================='''
def simulationChiSquarePayer(evalDate, N, S0, K, expiry, tenor, sigma1, sigma2, alpha1, alpha2, 
                              rho, eta, delta, simulation):
    # Find Zero Coupon Price at each date and calculate PVBP
    PVBP = np.sum(simulatedZeroCoupons[simulation][evalDate][expiry-1:tenor+expiry-1])
    
    # Parameters for input into Chi Square CDF
    sigmaSquaredT =  sigmaAlphaBetaVect(evalDate, expiry, tenor, sigma1, sigma2, 
                                    alpha1, alpha2, rho, eta, delta, simulation)
    T = expiry
    d = (np.power(K+delta,2 - 2*eta))/(np.power(1 - eta,2)* sigmaSquaredT)
    b = 1/(1 - eta)
    f = (np.power(S0+delta, 2 - 2*eta))/(np.power(1 - eta, 2)* sigmaSquaredT)

    # Calculate Price
    price = N * PVBP * ((S0+delta)*(1 - stats.ncx2.cdf(d, b+2, f)) - (K+delta)*(stats.ncx2.cdf(f, b, d)))
    return(price)

# Vectorize Function
simulationChiSquarePayerVect = np.vectorize(calibrationChiSquarePayer)

In [ ]:
swaptionPrices = []
for i in range(1000):
    swaptionPrices.append([simulationChiSquarePayerVect(np.arange(elem[1][0]), 1, elem[0], elem[0], 
                                    elem[1][0] - np.arange(elem[1][0]), elem[1][1], 
                                    sigma1, sigma2, 
                                    alpha1, alpha2, rho, eta, delta, i)
                                    for elem in list(zip(simulatedStrikes[i], swaptionMatrix))])


In [ ]:
simulatedStrikes[1]